# Import Packages

In [ ]:
import quantities as pq
import elephant

import sys
sys.path.append('..')
from modules.datasets import safe_download, MetadataSelector, LoadAndPrepareData
from modules.gui import EphyviewerConfigurator

# Select Data Set

The next cell will download `metadata.yml`, a file containing information about available data sets and how they should be loaded, prepared, and displayed, including:
* locations of data files
* filter parameters
* amplitude windows for spike discrimination
* video synchronization offset
* plot ranges

You can change these values to your liking. The file will only be downloaded if a copy isn't found locally to avoid overwritting local changes. If you want to redownload the file, you should delete your local copy first.

In [ ]:
# download the metadata YAML file if it isn't found locally
# - you need to manually delete this file if you want to redownload it
remote_metadata = 'ftp://slugcloud.case.edu/gill/Mirror of DynamicsHJC/Data/analysis/metadata.yml'
safe_download(remote_metadata, '../../data/metadata.yml')

The `MetadataSelector` allows you to view the names and descriptions of data sets listed in `metadata.yml` and to select the one you would like to work with. Each data set may contain multiple files, such as an electrophysiology data file and a video file. Here is the key for icons found at the left of each entry:<br/>
&nbsp;&nbsp; ◆ &nbsp;&nbsp; all files can be found locally and none need to be downloaded<br/>
&nbsp;&nbsp; ⬖ &nbsp;&nbsp; some files can be found locally and others cannot<br/>
&nbsp;&nbsp; ◇ &nbsp;&nbsp; no files can be found locally and all need to be downloaded<br/>
&nbsp;&nbsp;&nbsp; ! &nbsp;&nbsp;&nbsp; the `video_offset` parameter is not set, which can cause out-of-sync video playback

In [ ]:
# choose a data set to explore
metadata = MetadataSelector(
    file='../../data/metadata.yml',
    initial_selection='IN VIVO / JG08 / 2018-06-21 / 002',
)
display(metadata)

The next cell will download all of the data files associated with the selected data set. A file will only be downloaded if a copy isn't found locally to avoid overwritting local changes. If you want to redownload a file, you should delete your local copy first.

In [ ]:
# download the data files if they aren't found locally
# - you need to manually delete files if you want to redownload them
metadata.download_all_data_files()

# Load Data and Configure Ephyviewer

The next cell will use the settings contained in `metadata.yml` for the selected data set to do the following:
* read the electrophysiology data file
* apply filters to signals
* read annotations contained in CSV files
* run a simple spike detection algorithm using amplitude windows
* import spikes previously sorted by the `Spike Sorting` notebook
* calculate rectified area under the curve (RAUC) time series for each signal

When this is complete, a configuration widget will display that allows you to control which of the modular data viewers you would like the application to show. Click the "Launch" button to start up the application.

__Each time a new data set is selected using the `MetadataSelector`, you must rerun this cell.__

In [ ]:
# this cell must be rerun each time a new data set is selected above

# lazy loading takes advantage of Neo's high performance RawIO classes,
# which can read portions of files "on demand" to get just the data that
# is needed for plotting the currently visible time span
# - this decreases loading time and consumes much less memory, especially
#   for large files
# - to take advantage of these benefits, signal filtering, amplitude
#   window spike discrimination, and rauc computation must be disabled
# - spike markers on signals are currently incompatible with lazy loading
lazy = True

blk = LoadAndPrepareData(metadata, lazy=lazy)

# compute rectified area under the curve (RAUC) for each signal
# - these computations require loading the entire file now, so only
#   perform them if we're not using lazy loading
rauc_sigs = []
if not lazy:
    for sig in blk.segments[0].analogsignals:
        rauc = elephant.signal_processing.rauc(sig, bin_duration = 0.1*pq.s)
        rauc.name = sig.name + ' RAUC'
        rauc_sigs.append(rauc)

ephyviewer_config = EphyviewerConfigurator(metadata, blk, rauc_sigs, lazy)
# ephyviewer_config.enable_all()
display(ephyviewer_config)

Executing the next cell is equivalent to pressing the "Launch" button and is provided for the convenience of running all cells in the notebook at once to start the application.

In [ ]:
# using the buttons above, select the viewers you'd like to see,
# then either click "Launch" or run this cell
ephyviewer_config.launch_ephyviewer()